# ETL - Sprint #2

Desarrolla el tratamiento de los datos con el fin de preparar los datos para la siguiente etapa (visualización, machine learning, etc.)

# 1. Instalación de librerías

In [ ]:
# Dependencias
import os
import pandas as pd
import ast
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import folium
from folium.plugins import HeatMap
import imageio
import os
import geopandas as gpd
import contextily as ctx

from shapely.geometry import Point
from shapely.geometry import mapping

In [ ]:
!pip install ijson
!pip install pickle5

In [ ]:
import dask.dataframe as dd

import pyarrow as pa
import pyarrow.parquet as pq
import glob
import json
import pickle

# 2. Preparación y limpieza del dataset metadata_sitios de Google
<a id="1-Analisis-Google"></a>

## a. Junta y revisa todos los archivos json (metadata_sitios)

In [ ]:
# Define la ruta de la carpeta principal donde se encuentran los archivos JSON
main_folder = '/content/drive/MyDrive/AULATEC/DATASETS PF_HENRY/metadata-sitios'

# Busca todos los archivos JSON en la carpeta principal y subcarpetas
all_json_files = glob.glob(os.path.join(main_folder, '**', '*.json'), recursive=True)

# Cargar los archivos JSON en un DataFrame de Dask
ddf = dd.read_json(all_json_files, blocksize='64MB')  # Ajusta el blocksize según sea necesario

# Convertir a un DataFrame de pandas para análisis adicional
metadata_sitios = ddf.compute()

# Mostrar el DataFrame combinado
print(metadata_sitios)

In [ ]:
metadata_sitios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025011 entries, 0 to 3025010
Data columns (total 15 columns):
 #   Column            Dtype  
---  ------            -----  
 0   name              string 
 1   address           string 
 2   gmap_id           string 
 3   description       string 
 4   latitude          float64
 5   longitude         float64
 6   category          string 
 7   avg_rating        float64
 8   num_of_reviews    int64  
 9   price             string 
 10  hours             string 
 11  MISC              string 
 12  state             string 
 13  relative_results  string 
 14  url               string 
dtypes: float64(3), int64(1), string(11)
memory usage: 346.2 MB


In [ ]:
print(metadata_sitios.head())

               name                                            address  \
0   Porter Pharmacy  Porter Pharmacy, 129 N Second St, Cochran, GA ...   
1      City Textile  City Textile, 3001 E Pico Blvd, Los Angeles, C...   
2      San Soo Dang  San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
3      Nova Fabrics  Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...   
4  Nobel Textile Co  Nobel Textile Co, 719 E 9th St, Los Angeles, C...   

                                 gmap_id description   latitude   longitude  \
0  0x88f16e41928ff687:0x883dad4fd048e8f8        <NA>  32.388300  -83.357100   
1  0x80c2c98c0e3c16fd:0x29ec8a728764fdf9        <NA>  34.018891 -118.215290   
2  0x80c2c778e3b73d33:0xbdc58662a4a97d49        <NA>  34.058092 -118.292130   
3   0x80c2c89923b27a41:0x32041559418d447        <NA>  34.023669 -118.232930   
4  0x80c2c632f933b073:0xc31785961fe826a6        <NA>  34.036694 -118.249421   

                category  avg_rating  num_of_reviews price  \
0           ['Phar

## c. Segmenta por ubicación geográfica
Una vez hecho la segmentación, el nuevo dataset contiene 75,815 registros.

In [ ]:
# Cargar el archivo geojson que contiene los límites geográficos del condado de Los Angeles
gdf = gpd.read_file(r'G:\My Drive\HENRY\PROYECTO_FINAL\DATASETS\GEOJSON\Los_Angeles_polygon.geojson')

# Obtiene la primera geometría del archivo geojson
los_angeles_polygon = gdf['geometry'].iloc[0]

# Función para verificar si un punto (latitud, longitud) está dentro del polígono de Los Angeles County
def point_within_polygon(lat, lon, polygon):
    point = Point(lon, lat)
    return point.within(polygon)

# Aplicar la función para filtrar el DataFrame metadata_sitios, crea una nueva columna "within_la_county"
# para indicar si el regitro está o no (True/False) dentro del condado de Los Angeles
metadata_sitios['within_la_county'] = metadata_sitios.apply(lambda row: point_within_polygon(row['latitude'], row['longitude'], los_angeles_polygon), axis=1)

# Filtrar el DataFrame por puntos dentro del polígono de Los Angeles County (within_la_county = True)
la_county_df = metadata_sitios[metadata_sitios['within_la_county']]

# Mostrar el DataFrame resultante
print(la_county_df)


                                  name  \
1                         City Textile   
2                         San Soo Dang   
3                         Nova Fabrics   
4                     Nobel Textile Co   
5        Matrix International Textiles   
...                                ...   
3024707                          LADWP   
3024725                  See's Candies   
3024731                EZ 2 Rent A Car   
3024743     Armor Lock & Key, S.C.S.C.   
3024752            A&B Motor Cars Inc.   

                                                   address  \
1        City Textile, 3001 E Pico Blvd, Los Angeles, C...   
2        San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
3        Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...   
4        Nobel Textile Co, 719 E 9th St, Los Angeles, C...   
5        Matrix International Textiles, 1363 S Bonnie B...   
...                                                    ...   
3024707           LADWP, 6801 2nd St, Long Beach, CA 90803   

In [ ]:
la_county_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75815 entries, 1 to 3024752
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              75815 non-null  string 
 1   address           74729 non-null  string 
 2   gmap_id           75815 non-null  string 
 3   description       8890 non-null   string 
 4   latitude          75815 non-null  float64
 5   longitude         75815 non-null  float64
 6   category          75319 non-null  string 
 7   avg_rating        75815 non-null  float64
 8   num_of_reviews    75815 non-null  int64  
 9   price             8415 non-null   string 
 10  hours             60190 non-null  string 
 11  MISC              60945 non-null  string 
 12  state             60905 non-null  string 
 13  relative_results  69352 non-null  string 
 14  url               75815 non-null  string 
 15  within_la_county  75815 non-null  bool   
dtypes: bool(1), float64(3), int64(1), string(11

## d. Prepara el campo "category" para la siguiente segmentación
<a id="sección-4-Frecuencias-category-name"></a>

In [ ]:
''' Se observa que el campo "category" contiene listas y en algunos casos, listas de listas; sin embargo, el tipo de dato en este campo está definido como STRING, por lo que antes de explotarlo hay que convertilo a listas'''
# Función para asegurar que todos los elementos sean listas
def ensure_list(x):
    if isinstance(x, str):
        # Intenta convertir una cadena que parece una lista en una lista real
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return [x]
    elif not isinstance(x, list):
        return [x]
    return x

# Aplicar la función para asegurar que todos los elementos son listas
la_county_df['category'] = la_county_df['category'].apply(ensure_list)

# Verificar nuevamente los tipos después de la conversión
print(la_county_df['category'].apply(type).unique())

# Función para explotar completamente una columna específica
def fully_explode_column(df, column):
    while df[column].apply(lambda x: isinstance(x, list)).any():
        df = df.explode(column)
    return df

# Aplicar la función a la columna 'category'
df_exploded = fully_explode_column(la_county_df, 'category')

# Mostrar el DataFrame con la columna 'category' completamente explotada
print(df_exploded)


[<class 'list'>]
                                  name  \
1                         City Textile   
2                         San Soo Dang   
3                         Nova Fabrics   
4                     Nobel Textile Co   
5        Matrix International Textiles   
...                                ...   
3024731                EZ 2 Rent A Car   
3024731                EZ 2 Rent A Car   
3024731                EZ 2 Rent A Car   
3024743     Armor Lock & Key, S.C.S.C.   
3024752            A&B Motor Cars Inc.   

                                                   address  \
1        City Textile, 3001 E Pico Blvd, Los Angeles, C...   
2        San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
3        Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...   
4        Nobel Textile Co, 719 E 9th St, Los Angeles, C...   
5        Matrix International Textiles, 1363 S Bonnie B...   
...                                                    ...   
3024731  EZ 2 Rent A Car, 3290 Cherry Ave, L

## e. Segmenta los comercios de Los Ángeles, conserva solo los restaurantes
<a id="sección-6-top10-negocios-categorias"></a>

In [ ]:
# Para poder aplicar el fitro por tipo de reubro (category) es necesario eliminar los valores nulos.

# Elimina las 496 filas con valores nulos en el campo de category (representan el 0.3% del dataset df_exploded)
google_restaurants_la = df_exploded.dropna(subset=['category'])

# Elimina la columna 'within_la_county' porque ya no se necesita
google_restaurants_la = google_restaurants_la.drop(columns= 'within_la_county')

# Filtrar el DataFrame para incluir solo filas donde 'category' contiene la palabra 'restaurant' (ignorar mayúsculas/minúsculas)
google_restaurants_la = google_restaurants_la[google_restaurants_la['category'].str.contains('restaurant', case=False, regex=True)]


# Mostrar el DataFrame resultante posee 13,955 registros (Restaurantes en el condado de Los Angeles)
google_restaurants_la.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13955 entries, 2 to 3024600
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              13955 non-null  string 
 1   address           13918 non-null  string 
 2   gmap_id           13955 non-null  string 
 3   description       6668 non-null   string 
 4   latitude          13955 non-null  float64
 5   longitude         13955 non-null  float64
 6   category          13955 non-null  object 
 7   avg_rating        13955 non-null  float64
 8   num_of_reviews    13955 non-null  int64  
 9   price             8355 non-null   string 
 10  hours             13064 non-null  string 
 11  MISC              13902 non-null  string 
 12  state             13070 non-null  string 
 13  relative_results  11874 non-null  string 
 14  url               13955 non-null  string 
dtypes: float64(3), int64(1), object(1), string(10)
memory usage: 1.7+ MB


In [ ]:
# Exporta a un archivo json la información de lo restaurantes en Los Angeles
google_restaurants_la.to_json('G:\My Drive\HENRY\PROYECTO_FINAL\DATASETS\JSON\google_restaurants_la.json', orient='records', lines=True)

# 3. Relaciona el data set de reviews de California (reviews_california) con el de sitios (ambos de Google Maps)

In [ ]:
# Define la ruta de la carpeta principal donde se encuentran los archivos JSON (18 archivos)
main_folder = '/content/drive/MyDrive/AULATEC/DATASETS PF_HENRY/reviews-estados/review-California'

# Busca todos los archivos JSON en la carpeta principal y subcarpetas
all_json_files = glob.glob(os.path.join(main_folder, '**', '*.json'), recursive=True)

# Lista para almacenar los DataFrames
dataframes = []

# Función para leer y procesar archivos JSON
def read_json_file(file_path):
    data = []
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            # Intenta cargar todo el contenido del archivo como un solo objeto JSON
            try:
                data = json.loads(content)
                if not isinstance(data, list):
                    data = [data]
            except json.JSONDecodeError:
                # Si falla, intenta cargar línea por línea
                lines = content.splitlines()
                for line in lines:
                    try:
                        data.append(json.loads(line))
                    except json.JSONDecodeError:
                        continue
    except Exception as e:
        print(f"Error reading file {file_path}, error: {e}")
    return data

# Leer y cargar cada archivo JSON en un DataFrame de pandas
for json_file in all_json_files:
    data = read_json_file(json_file)
    if data:
        df = pd.DataFrame(data)
        dataframes.append(df)

# Combinar todos los DataFrames en uno solo (opcional, si es que tienen la misma estructura)
combined_df = pd.concat(dataframes, ignore_index=True)

# Mostrar el DataFrame combinado
print(combined_df)


In [74]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 164.8+ MB


In [78]:
combined_df.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,112640357449611959087,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,117440349723823658676,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,100580770836123539210,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [75]:
# Realizar la unión de los DataFrames en el campo 'gmap_id'
gmap_meta_reviews = pd.merge(google_restaurants_la, combined_df, on='gmap_id', how='inner')

# Mostrar el DataFrame resultante
gmap_meta_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517513 entries, 0 to 517512
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name_x            517513 non-null  string 
 1   address           517513 non-null  string 
 2   gmap_id           517513 non-null  object 
 3   description       424501 non-null  string 
 4   latitude          517513 non-null  float64
 5   longitude         517513 non-null  float64
 6   category          517513 non-null  object 
 7   avg_rating        517513 non-null  float64
 8   num_of_reviews    517513 non-null  int64  
 9   price             460548 non-null  string 
 10  hours             506538 non-null  string 
 11  MISC              517392 non-null  string 
 12  state             506737 non-null  string 
 13  relative_results  503399 non-null  string 
 14  url               517513 non-null  string 
 15  user_id           517513 non-null  object 
 16  name_y            51

In [76]:
gmap_meta_reviews.head()

,name_x,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,...,state,relative_results,url,user_id,name_y,time,rating,text,pics,resp
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,<NA>,34.058092,-118.29213,Korean restaurant,4.4,18,<NA>,...,Open ⋅ Closes 6PM,"['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,None,None
1,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,<NA>,34.058092,-118.29213,Korean restaurant,4.4,18,<NA>,...,Open ⋅ Closes 6PM,"['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,None,None
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,<NA>,34.058092,-118.29213,Korean restaurant,4.4,18,<NA>,...,Open ⋅ Closes 6PM,"['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,112640357449611959087,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None
3,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,<NA>,34.058092,-118.29213,Korean restaurant,4.4,18,<NA>,...,Open ⋅ Closes 6PM,"['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,117440349723823658676,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None
4,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,<NA>,34.058092,-118.29213,Korean restaurant,4.4,18,<NA>,...,Open ⋅ Closes 6PM,"['0x80c2c78249aba68f:0x35bf16ce61be751d', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,100580770836123539210,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None


In [80]:
#  Exporta a un archivo json la nueva tabla (metadatada_sitios + Reviews)
gmap_meta_reviews.to_json('G:\My Drive\HENRY\PROYECTO_FINAL\DATASETS\JSON\gmap_meta_reviews.json', orient='records', lines=True)